In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Import_Export_Citric_Acid_Data.xlsx', sheet_name = 'Raw Data')
df.head()

In [ ]:
df[["Quantity", "Value"]] = df[["Quantity", "Value"]].astype(float)
df.head()

In [ ]:
# delete ceros from the dataset in columns Quantity and Value
df = df.replace(0, np.nan)
df = df.dropna(subset=["Quantity", "Value"])

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

# Market Demand: Which countries are exhibiting high demand for citric acid?

In [ ]:
# Market Demand: Which countries are exhibiting high demand for citric acid?
df_import = df[df['Trade Direction'] == 'IMPORT']
df_import

In [ ]:
df_import['Buyer Country'].value_counts(sort = True).head(12)

In [ ]:
df_3 = df_import['Buyer Country'].value_counts(normalize = True) * 100
df_3[df_3.cumsum() <= 90]

In [ ]:
pd.set_option('display.float_format', '{:,.2f}'.format)
df_4 = df_import.groupby('Buyer Country')[['Quantity', 'Value']].sum().sort_values(by='Value', ascending=False)
df_4['Cum_Value'] = df_4['Value'].cumsum()
df_4['Cum_Quantity'] = df_4['Quantity'].cumsum()
df_4

# Competitive Landscape: Who are the major players in these markets and what does their market share look like?

In [ ]:
# countries with higher number of buy for citrid acid are usa, mexico, india, costa rica and vietnam
# countries with higher number of quantity and value citrid acid imported are usa, india, mexico, canada and argentina

df_landscape = df_import[['Buyer', 'Buyer Country', 'Trade Direction','Quantity','Value']]
df_landscape.head()

In [ ]:
# 1. group by country and buyer
pivot = (
    df_landscape
    .groupby(["Buyer Country", "Buyer"], as_index=False)[["Quantity", "Value"]]
    .sum()
)

# 2. calculate total by country
totales = pivot.groupby("Buyer Country")[["Quantity", "Value"]].transform("sum")

# 3. market share
pivot["Market Share (Quantity)"] = (pivot["Quantity"] / totales["Quantity"] * 100).round(2)
pivot["Market Share (Value)"] = (pivot["Value"] / totales["Value"] * 100).round(2)

# 4. Order by country y participation (value) Desc
pivot = pivot.sort_values(
    by=["Buyer Country", "Market Share (Value)"],
    ascending=[True, False]
).reset_index(drop=True)

# 5. show all the dataframe
pd.set_option("display.max_rows", None)   # muestra todas las filas
pd.set_option("display.max_columns", None)  # muestra todas las columnas
pd.set_option("display.expand_frame_repr", False)  # evita particionar en bloques

pivot


In [ ]:
pivot_filtered = pivot[pivot["Buyer Country"].isin(["UNITED STATES OF AMERICA", "INDIA", 'MEXICO', 'CANADA','ARGENTINA'])]
pivot_filtered.loc[:, 'cum_sum_share_units'] = (pivot_filtered.groupby("Buyer Country")["Market Share (Quantity)"].cumsum())
pivot_filtered.loc[:, 'cum_sum_share_value'] = (pivot_filtered.groupby("Buyer Country")["Market Share (Value)"].cumsum())
pivot_filtered

# Pricing Strategy: What insights does the data provide about average product prices in different markets, and how can we leverage this information to develop competitive pricing strategies?

In [ ]:
df_import['Average_price'] = df_import['Value'] / df_import['Quantity']
df_import

In [ ]:
df_import.groupby('Buyer Country')['Average_price'].mean().sort_values(ascending = False)

In [ ]:
# Calculate unit price and prepare data
df = df.copy()

# Calculate unit price (Value / Quantity)
df["Unit Price"] = df["Value"] / df["Quantity"]
df["Date"] = pd.to_datetime(df["Date"])
df.head()

In [ ]:
# General price analysis by buyer country

price_by_country = (
    df.groupby("Buyer Country")["Unit Price"]
    .mean()
    .sort_values(ascending=False)
)

print(price_by_country)

In [ ]:
# Compare suppliers within a specific country

price_by_supplier = (
    df[df["Buyer Country"] == "INDIA"]
    .groupby("Supplier")["Unit Price"]
    .mean()
    .sort_values(ascending=False)
)

print(price_by_supplier)

In [ ]:
# List of Buyer Countries to compare
countries = ["INDIA", "UNITED STATES OF AMERICA", "MEXICO", "CANADA", "ARGENTINA"]

plt.figure(figsize=(12, 6))

for country in countries:
    # Filter by Buyer Country
    subset = df[df["Buyer Country"] == country]

    # Group by month and calculate average unit price
    price_trend = (
        subset.groupby(pd.Grouper(key="Date", freq="M"))["Unit Price"]
        .mean()
    )

    # Plot line for each country
    plt.plot(price_trend.index, price_trend.values, marker="o", label=country)

# Chart details
plt.title("Monthly Unit Price Evolution by Buyer Country")
plt.xlabel("Date")
plt.ylabel("Unit Price")
plt.legend(title="Buyer Country")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Buyer benchmark (market share in a country)

# List of Buyer Countries to analyze
countries = ["UNITED STATES OF AMERICA", "INDIA", "MEXICO", "CANADA", "ARGENTINA"]

for country in countries:
    print(f"\n--- Market Share of Buyers in {country} ---\n")

    # Filter by Buyer Country
    buyers_share = (
        df[df["Buyer Country"].str.upper() == country.upper()]
        .groupby("Buyer")[["Quantity", "Value"]]
        .sum()
        .sort_values("Value", ascending=False)
    )

    # Calculate market share (%)
    buyers_share["Market Share (Value)"] = (
        buyers_share["Value"] / buyers_share["Value"].sum() * 100
    ).round(2)

    # Show Top 10 Buyers
    print(buyers_share.head(10))



In [ ]:
# Countries of interest
countries = ["UNITED STATES OF AMERICA", "INDIA", "MEXICO", "CANADA", "ARGENTINA"]

# Filter the DataFrame
df_filtered = df[df["Buyer Country"].str.upper().isin([c.upper() for c in countries])]

# Plot
plt.figure(figsize=(8,5))
sns.boxplot(data=df_filtered, x="Buyer Country", y="Unit Price")
plt.xticks(rotation=45)
plt.title("Unit Price Distribution by Buyer Country")
plt.show()
